In [1]:
import numpy as np
import pandas as pd
import os
import requests
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Extracting the Stanford Feature Set

In [2]:
maindir = os.getcwd()
os.chdir("/../../datasets/aclImdb/")

In [3]:
voc_f = open("imdb.vocab",'r')
vocab = voc_f.read().split("\n")

In [4]:
Exp_f = open("imdbEr.txt",'r')
EV = Exp_f.read().split("\n")[:-1]

In [5]:
vocabInfo = pd.DataFrame({"vocab":vocab,"expected":EV})
vocabInfo

,vocab,expected
0,the,0.0490972013402
1,and,0.201363575849
2,a,0.0333946807184
3,of,0.099837669572
4,to,-0.0790210365788
...,...,...
89522,copywrite,0
89523,artbox,0
89524,kinky-sex,0
89525,urrrghhh,0


In [6]:
en_stops = set(stopwords.words('english'))

vocabInfo = vocabInfo[~ vocabInfo["vocab"].isin(en_stops)]
vocabInfo["expected"] = abs(vocabInfo["expected"].astype(float))
vocabInfo

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,vocab,expected
15,movie,0.246354
17,film,0.147886
26,one,0.087114
27,!,0.134839
36,like,0.280548
...,...,...
89522,copywrite,0.000000
89523,artbox,0.000000
89524,kinky-sex,0.000000
89525,urrrghhh,0.000000


In [7]:
vocabInfo.sort_values(by="expected",ascending=False)[:round(len(vocabInfo)*0.05)]

,vocab,expected
54867,pelswick,4.500000
33943,magorian,4.500000
67306,subspace,4.500000
35092,jgar,4.500000
58224,buh,4.500000
...,...,...
81702,danira,2.267657
61418,ecclesiastical,2.267421
40382,grey-haired,2.267421
44828,irland,2.267421


In [8]:
#sub = (vocabInfo[vocabInfo["expected"] <= 2].sort_values(by="expected"))
#sub_vocab = list(sub)

allowed_word_types = ["J"]

adjectives = []
# parts of speech tagging for each word 
pos = nltk.pos_tag(list(vocabInfo["vocab"]))
    
# make a list of  all adjectives identified by the allowed word types list above
for w in pos:
    if w[1][0] in allowed_word_types:
        adjectives.append(w[0].lower())   

#reduced_vocab = pd.Series(reduced_vocab)
#reduced_vocab = reduced_vocab.sample(n=round(len(reduced_vocab)*0.3),random_state=1)
len(adjectives)

30896

In [9]:
# Positive semantic words
url = "https://raw.githubusercontent.com/jeffreybreen/twitter-sentiment-analysis-tutorial-201107/master/data/opinion-lexicon-English/positive-words.txt"
r = requests.get(url)
s=';;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n; \n; Opinion Lexicon: Positive\n;\n; This file contains a list of POSITIVE opinion words (or sentiment words).\n;\n; This file and the papers can all be downloaded from \n;    http://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html\n;\n; If you use this list, please cite one of the following two papers:\n;\n;   Minqing Hu and Bing Liu. "Mining and Summarizing Customer Reviews." \n;       Proceedings of the ACM SIGKDD International Conference on Knowledge \n;       Discovery and Data Mining (KDD-2004), Aug 22-25, 2004, Seattle, \n;       Washington, USA, \n;   Bing Liu, Minqing Hu and Junsheng Cheng. "Opinion Observer: Analyzing \n;       and Comparing Opinions on the Web." Proceedings of the 14th \n;       International World Wide Web conference (WWW-2005), May 10-14, \n;       2005, Chiba, Japan.\n;\n; Notes: \n;    1. The appearance of an opinion word in a sentence does not necessarily  \n;       mean that the sentence expresses a positive or negative opinion. \n;       See the paper below:\n;\n;       Bing Liu. "Sentiment Analysis and Subjectivity." An chapter in \n;          Handbook of Natural Language Processing, Second Edition, \n;          (editors: N. Indurkhya and F. J. Damerau), 2010.\n;\n;    2. You will notice many misspelled words in the list. They are not \n;       mistakes. They are included as these misspelled words appear \n;       frequently in social media content. \n;\n;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;'
pos_words = r.text[len(s)+2:]
pos_words = pos_words.split("\n")

# Negative semantic words
url = "https://raw.githubusercontent.com/jeffreybreen/twitter-sentiment-analysis-tutorial-201107/master/data/opinion-lexicon-English/negative-words.txt"
r = requests.get(url)
s=';;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n; \n; Opinion Lexicon: Negative\n;\n; This file contains a list of NEGATIVE opinion words (or sentiment words).\n;\n; This file and the papers can all be downloaded from \n;    http://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html\n;\n; If you use this list, please cite one of the following two papers:\n;\n;   Minqing Hu and Bing Liu. "Mining and Summarizing Customer Reviews." \n;       Proceedings of the ACM SIGKDD International Conference on Knowledge \n;       Discovery and Data Mining (KDD-2004), Aug 22-25, 2004, Seattle, \n;       Washington, USA, \n;   Bing Liu, Minqing Hu and Junsheng Cheng. "Opinion Observer: Analyzing \n;       and Comparing Opinions on the Web." Proceedings of the 14th \n;       International World Wide Web conference (WWW-2005), May 10-14, \n;       2005, Chiba, Japan.\n;\n; Notes: \n;    1. The appearance of an opinion word in a sentence does not necessarily  \n;       mean that the sentence expresses a positive or negative opinion. \n;       See the paper below:\n;\n;       Bing Liu. "Sentiment Analysis and Subjectivity." An chapter in \n;          Handbook of Natural Language Processing, Second Edition, \n;          (editors: N. Indurkhya and F. J. Damerau), 2010.\n;\n;    2. You will notice many misspelled words in the list. They are not \n;       mistakes. They are included as these misspelled words appear \n;       frequently in social media content. \n;\n;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n\n'
neg_words = r.text[len(s):]
neg_words = neg_words.split("\n")

all_words = pos_words + neg_words

In [10]:
# get all adjectives that have a "good" or "bad" association
word_features = []
for w in adjectives:
    if w in all_words:
        word_features.append(w)
len(word_features)

1900

# Using the Derived Feature Set

In [11]:
os.chdir(maindir)

df_train = pd.read_csv("reviews_train.csv").drop("Unnamed: 0", axis=1)
df_test = pd.read_csv("reviews_test.csv").drop("Unnamed: 0", axis=1)

## Train

In [12]:
# function to create a dictionary of features for each review in the list document.
# The keys are the words in word_features 
# The values of each key are either true or false for wether that feature appears in the review or not
def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

# Creating features for each review
featuresets = []

for i, row in df_train.iterrows():
    featuresets.append((find_features(row["Review"]), row["Label"]))

In [13]:
df = pd.DataFrame.from_dict(featuresets)
df_train = pd.json_normalize(df[0])
df_train = df_train.astype(int)
df_train["Label"] = df[1]
df_train.head()

,good,bad,great,best,better,funny,enough,right,worst,hard,...,halcyon,swiftness,semi-retarded,inviolable,impolite,unlicensed,unconstitutional,uneasily,last-ditch,Label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


## Test

In [14]:
for i, row in df_test.iterrows():
    featuresets.append((find_features(row["Review"]), row["Label"]))
    
df2 = pd.DataFrame.from_dict(featuresets)
df2.head()

df_test = pd.json_normalize(df[0])
df_test = df_train.astype(int)
df_test["Label"] = df2[1]
df_test.head()

,good,bad,great,best,better,funny,enough,right,worst,hard,...,halcyon,swiftness,semi-retarded,inviolable,impolite,unlicensed,unconstitutional,uneasily,last-ditch,Label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


## Saving Train and Test DataFrames

In [15]:
df_train.to_csv("stanford_train2.csv")
df_train.to_csv("stanford_test2.csv")